![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises
https://pgexercises.com/ <br>
https://pgexercises.com/gettingstarted.html

In [ ]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-01-16 09:28:58.927360+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Install & Configure Postgres

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.11 (Ubuntu 14.11-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
#!pip install psycopg2
import psycopg2
import pandas as pd

#Create Schema, Tables and Insert Data

In [4]:
# Change the database superuser 'postgres' to a non-default value 'pass'
#alterseries command - DCL Ccommand
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"

ALTER ROLE


In [5]:
!wget -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/clubdata.sql

In [6]:
# Download and execute SQL file to create tables, load data
#sql is a lonf text file which can contain 20k commands and as long as they are correct it will run all of them. First we get the command using Wget.
#export file-
#-f means file after this and instruct sql to use that
#!wget https://pgexercises.com/dbfiles/clubdata.sql
#!wget -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/clubdata.sql
#Run the batch file to create tables, load data
!psql -h localhost -U postgres -f clubdata.sql -d postgres -x -q


Password for user postgres: 


In [7]:
# Create file to store the password. Else will be prompted each time
#!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!echo "localhost:5432:exercises:postgres:pass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:exercises:postgres:pass


In [8]:
#Note new database called exercises
#\l - to show the database.
#here we are trying to find out the databases but we haven't connected to exercises database yet

!psql -h localhost -p 5432 -Upostgres -c "\l"

Password for user postgres: 
                                  List of databases
   Name    |  Owner   | Encoding |   Collate   |    Ctype    |   Access privileges   
-----------+----------+----------+-------------+-------------+-----------------------
 exercises | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 postgres  | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 template0 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
 template1 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
(4 rows)



In [9]:
#Note three tables under schema cd
#now we are connecting to exercise database and asking to show the table
#cd.* = In sql file schema called cd and created three tables under the schema so all tables begin with qualifier cd.
#dt+ is list of tables
!psql -h localhost -p 5432 -Upostgres -d exercises -c "\dt+ cd.*"
#!psql -h localhost -p 5432 -Upostgres -d exercises -c "\dt+ *.*" # gives all tables with different schemas

                                      List of relations
 Schema |    Name    | Type  |  Owner   | Persistence | Access method |  Size  | Description 
--------+------------+-------+----------+-------------+---------------+--------+-------------
 cd     | bookings   | table | postgres | permanent   | heap          | 264 kB | 
 cd     | facilities | table | postgres | permanent   | heap          | 16 kB  | 
 cd     | members    | table | postgres | permanent   | heap          | 16 kB  | 
(3 rows)



## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [10]:
# Local Colab server

conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.11 (Ubuntu 14.11-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [11]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

##Check Data

In [12]:
runSelect("select * from cd.members limit 3")

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,None,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,None,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,None,2012-07-02 12:08:23


In [13]:
runSelect("select * from cd.facilities limit 3")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50


In [14]:
runSelect("select * from cd.bookings limit 3")

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [15]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2024-02-29 14:46:14.796221+05:30


#Basics


In [16]:
#1 How can you retrieve all the information from the cd.facilities table?
runSelect("select * from cd.facilities")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50
3,Table Tennis,0,5,320,10
4,Massage Room 1,35,80,4000,3000
5,Massage Room 2,35,80,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0,5,450,15
8,Pool Table,0,5,400,15


In [17]:
#2 You want to print out a list of all of the facilities and their cost to members. How would you retrieve a list of only facility names and costs?
runSelect("select name, membercost from cd.facilities ")

name,membercost
Tennis Court 1,5
Tennis Court 2,5
Badminton Court,0
Table Tennis,0
Massage Room 1,35
Massage Room 2,35
Squash Court,3.5
Snooker Table,0
Pool Table,0


In [18]:
#3 How can you produce a list of facilities that charge a fee to members?
runSelect("select name, membercost, guestcost, initialoutlay, monthlymaintenance \
from cd.facilities \
where membercost > 0")

name,membercost,guestcost,initialoutlay,monthlymaintenance
Tennis Court 1,5,25,10000,200
Tennis Court 2,5,25,8000,200
Massage Room 1,35,80,4000,3000
Massage Room 2,35,80,4000,3000
Squash Court,3.5,17.5,5000,80


In [19]:
#4 How can you produce a list of facilities that charge a fee to members, and that fee is less than 1/50th of the monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.
runSelect("select facid, name, membercost, monthlymaintenance \
from cd.facilities \
where membercost > 0 and (membercost < monthlymaintenance/50.0)")

facid,name,membercost,monthlymaintenance
4,Massage Room 1,35,3000
5,Massage Room 2,35,3000


In [20]:
#Basic string searches
#5 How can you produce a list of all facilities with the word 'Tennis' in their name?
runSelect("select facid, name, membercost, guestcost, initialoutlay, monthlymaintenance \
from cd.facilities \
where name like '%Tennis%' ")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
3,Table Tennis,0,5,320,10


In [21]:
#Matching against multiple possible values
#6 How can you retrieve the details of facilities with ID 1 and 5? Try to do it without using the OR operator.
runSelect("select facid, name, membercost, guestcost, initialoutlay, monthlymaintenance \
from cd.facilities \
where facid in (1,5) ")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5,25,8000,200
5,Massage Room 2,35,80,4000,3000


In [23]:
#Classify results into buckets
#7) How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive' depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.
runSelect("select name, \
case when monthlymaintenance >100 then 'expensive' else 'cheap' end as cost from cd.facilities")

name,cost
Tennis Court 1,expensive
Tennis Court 2,expensive
Badminton Court,cheap
Table Tennis,cheap
Massage Room 1,expensive
Massage Room 2,expensive
Squash Court,cheap
Snooker Table,cheap
Pool Table,cheap


In [24]:
#Working with dates
#8 How can you produce a list of members who joined after the start of September 2012? Return the memid, surname, firstname, and joindate of the members in question.
runSelect("select memid, surname, firstname, joindate \
from cd.members \
where joindate >= '2012-09-01' ")

memid,surname,firstname,joindate
24,Sarwin,Ramnaresh,2012-09-01 08:44:42
26,Jones,Douglas,2012-09-02 18:43:05
27,Rumney,Henrietta,2012-09-05 08:42:35
28,Farrell,David,2012-09-15 08:22:05
29,Worthington-Smyth,Henry,2012-09-17 12:27:15
30,Purview,Millicent,2012-09-18 19:04:01
33,Tupperware,Hyacinth,2012-09-18 19:32:05
35,Hunt,John,2012-09-19 11:32:45
36,Crumpet,Erica,2012-09-22 08:36:38
37,Smith,Darren,2012-09-26 18:08:45


In [25]:
#Removing duplicates, and ordering results
#9 How can you produce an ordered list of the first 10 surnames in the members table? The list must not contain duplicates.
runSelect("select distinct surname from cd.members order by surname limit 10 ")

surname
Bader
Baker
Boothe
Butters
Coplin
Crumpet
Dare
Farrell
Genting
GUEST


In [26]:
#Combining results from multiple queries
#10 You, for some reason, want a combined list of all surnames and all facility names. Yes, this is a contrived example :-). Produce that list
runSelect("select surname from cd.members union select name from cd.facilities")

surname
Hunt
Farrell
Tennis Court 2
Table Tennis
Dare
Rownam
GUEST
Badminton Court
Smith
Tupperware


In [27]:
#Simple aggregation
#11 You'd like to get the signup date of your last member. How can you retrieve this information?
runSelect("select max(joindate)as latest from cd.members")

latest
2012-09-26 18:08:45


In [28]:
#more aggregation
#12 You'd like to get the first and last name of the last member(s) who signed up - not just the date. How can you do that?

In [29]:
runSelect("select firstname, surname, joindate from cd.members where joindate = (select max(joindate) from cd.members)")

firstname,surname,joindate
Darren,Smith,2012-09-26 18:08:45


# Joins and Subqueries

In [30]:
#Retrieve the start times of members' bookings
#1 How can you produce a list of the start times for bookings by members named 'David Farrell'?
runSelect("select * from cd.bookings limit 5")

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2
3,7,1,2012-07-03 19:00:00,2
4,8,1,2012-07-03 10:00:00,1


In [31]:
#Retrieve the start times of members' bookings
#2 How can you produce a list of the start times for bookings by members named 'David Farrell'?
runSelect("select booking.starttime from cd.bookings booking \
inner join cd.members mem \
on mem.memid = booking.memid \
where mem.firstname = 'David' and mem.surname = 'Farrell' limit 5")

starttime
2012-09-18 09:00:00
2012-09-18 13:30:00
2012-09-18 17:30:00
2012-09-18 20:00:00
2012-09-19 09:30:00


In [32]:
#Work out the start times of bookings for tennis courts
#3 How can you produce a list of the start times for bookings for tennis courts, for the date '2012-09-21'? Return a list of start time and facility name pairings, ordered by the time.
runSelect("select booking.starttime start, facs.name name \
from cd.facilities facs \
inner join cd.bookings booking  \
on facs.facid = booking.facid \
where facs.name like '%Tennis Court 1%' and booking.starttime >= '2012-09-21' and booking.starttime < '2012-09-22' order by starttime")

start,name
2012-09-21 08:00:00,Tennis Court 1
2012-09-21 09:30:00,Tennis Court 1
2012-09-21 12:00:00,Tennis Court 1
2012-09-21 13:30:00,Tennis Court 1
2012-09-21 15:30:00,Tennis Court 1
2012-09-21 17:00:00,Tennis Court 1


In [33]:
#4
runSelect("select distinct recs.firstname firstname, recs.surname surname \
from cd.members mems \
inner join cd.members recs \
on recs.memid = mems.recommendedby \
order by surname, firstname")

firstname,surname
Florence,Bader
Timothy,Baker
Gerald,Butters
Jemima,Farrell
Matthew,Genting
David,Jones
Janice,Joplette
Millicent,Purview
Tim,Rownam
Darren,Smith


In [33]:
#5
runSelect("select mems.firstname memfname, mems.surname memsname, recs.firstname recfname, recs.surname recsname \
from cd.members mems \
left outer join cd.members recs \
on recs.memid = mems.recommendedby \
order by memsname, memfname")

memfname,memsname,recfname,recsname
Florence,Bader,Ponder,Stibbons
Anne,Baker,Ponder,Stibbons
Timothy,Baker,Jemima,Farrell
Tim,Boothe,Tim,Rownam
Gerald,Butters,Darren,Smith
Joan,Coplin,Timothy,Baker
Erica,Crumpet,Tracy,Smith
Nancy,Dare,Janice,Joplette
David,Farrell,None,None
Jemima,Farrell,None,None


In [34]:
#Question
##6 How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.
runSelect("select mems.firstname ||' '|| mems.surname member, facs.name facility \
		  from cd.members mems \
		  inner join cd.bookings bks \
		  on mems.memid = bks.memid \
		  inner join cd.facilities facs \
		  on bks.facid = facs.facid \
		  where facs.name in ('Tennis Court 2','Tennis Court 1') \
		  order by member, facility ")

member,facility
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 2
Anne Baker,Tennis Court 2
Anne Baker,Tennis Court 2
Anne Baker,Tennis Court 2


In [55]:
#Produce a list of costly bookings
#7 How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user is always ID 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.
runSelect("select mems.firstname || ' ' || mems.surname as member, \
facs.name as facility, case when mems.memid = 0 then bks.slots*facs.guestcost \
else bks.slots*facs.membercost \
end as cost from cd.members mems \
inner join cd.bookings bks on mems.memid = bks.memid \
inner join cd.facilities facs on bks.facid = facs.facid where \
bks.starttime >= '2012-09-14' and \
bks.starttime < '2012-09-15' and ((mems.memid = 0 and bks.slots*facs.guestcost > 30) or \
(mems.memid != 0 and bks.slots*facs.membercost > 30)) \
order by cost desc")

member,facility,cost
GUEST GUEST,Massage Room 2,320
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Tennis Court 2,150
Jemima Farrell,Massage Room 1,140
GUEST GUEST,Tennis Court 1,75
GUEST GUEST,Tennis Court 2,75
GUEST GUEST,Tennis Court 1,75
Matthew Genting,Massage Room 1,70


In [36]:
#Produce a list of all members, along with their recommender, using no joins.
#8 How can you output a list of all members, including the individual who recommended them (if any), without using any joins? Ensure that there are no duplicates in the list, and that each firstname + surname pairing is formatted as a column and ordered.
runSelect("select distinct mems.firstname ||''|| mems.surname member, \
(select recs.firstname ||''|| recs.surname recommender \
from cd.members recs \
where recs.memid = mems.recommendedby) \
from cd.members mems order by member")

member,recommender
AnnaMackenzie,DarrenSmith
AnneBaker,PonderStibbons
BurtonTracy,None
CharlesOwen,DarrenSmith
DarrenSmith,None
DavidFarrell,None
DavidJones,JaniceJoplette
DavidPinker,JemimaFarrell
DouglasJones,DavidJones
EricaCrumpet,TracySmith


# Modifying Data

In [37]:
#1 Insert some data into a table
#The club is adding a new facility - a spa. We need to add it into the facilities table. Use the following values:
#facid: 9, Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.
runCmd("insert into cd.facilities \
    (facid, name, membercost, guestcost, initialoutlay, monthlymaintenance) \
    values (9, 'Spa', 20, 30, 100000, 800)")

Command executed


In [38]:
#2 Insert multiple rows of data into a table
#In the previous exercise, you learned how to add a facility. Now you're going to add multiple facilities in one command. Use the following values:
#facid: 9, Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.
#facid: 10, Name: 'Squash Court 2', membercost: 3.5, guestcost: 17.5, initialoutlay: 5000, monthlymaintenance: 80.
runCmd(("insert into cd.facilities \
(facid, name, membercost, guestcost, initialoutlay, monthlymaintenance) \
values (10, 'Spa', 20, 30, 100000, 800), (11, 'Squash Court 2', 3.5, 17.5, 5000, 80)"))

Command executed


In [58]:
runSelect("select * from cd.facilities")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50
3,Table Tennis,0,5,320,10
4,Massage Room 1,35,80,4000,3000
5,Massage Room 2,35,80,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0,5,450,15
8,Pool Table,0,5,400,15
9,Spa,20,30,100000,800


In [69]:
#3 Insert calculated data into a table
#Let's try adding the spa to the facilities table again. This time, though, we want to automatically generate the value for the next facid, rather than specifying it as a constant. Use the following values for everything else:

#Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.
runCmd("insert into cd.facilities (facid, name, membercost, guestcost, initialoutlay, monthlymaintenance) \
select(max(facid) from cd.facilities)+1, 'Spa', 20, 30, 100000, 800")
runSelect("select * from cd.facilities")

Command failed


facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50
3,Table Tennis,0,5,320,10
4,Massage Room 1,35,80,4000,3000
5,Massage Room 2,35,80,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0,5,450,15
8,Pool Table,0,5,400,15


In [40]:
#4 Update some existing data
#Question
#We made a mistake when entering the data for the second tennis court. The initial outlay was 10000 rather than 8000: you need to alter the data to fix the error.
runCmd("update cd.facilities \
    set initialoutlay = 10000 \
    where facid = 1;")

Command executed


In [41]:
#5 We want to increase the price of the tennis courts for both members and guests. Update the costs to be 6 for members, and 30 for guests.
runCmd("update cd.facilities \
set membercost = 6, guestcost = 30 \
where name = 'Tennis Court%'")

Command executed


In [42]:
#6 Question
#We want to alter the price of the second tennis court so that it costs 10% more than the first one. Try to do this without using constant values for the prices, so that we can reuse the statement if we want to.
runCmd("update cd.facilities \
set (membercost = (select membercost * 1.1 from cd.facilities where facid = 0)) \
set (guestcost = (select guestcost * 1.1 from cd.facilities where facid = 0)) \
where facs.facid = 1;")

Command failed


In [43]:
runSelect("select * from cd.facilities")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
2,Badminton Court,0,15.5,4000,50
3,Table Tennis,0,5,320,10
4,Massage Room 1,35,80,4000,3000
5,Massage Room 2,35,80,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0,5,450,15
8,Pool Table,0,5,400,15
9,Spa,20,30,100000,800
10,Spa,20,30,100000,800


In [ ]:
#7 As part of a clearout of our database, we want to delete all bookings from the cd.bookings table. How can we accomplish this?
#runCmd("delete from cd.bookings")

In [ ]:
#8 We want to remove member 37, who has never made a booking, from our database. How can we achieve that?
#runCmd("delete from cd.bookings \
#where memid = 37")

In [ ]:
#9 In our previous exercises, we deleted a specific member who had never made a booking. How can we make that more general, to delete all members who have never made a booking?
#runCmd("delete from cd.members where memid not in (select memid from cd.bookings)")

# Aggregates

In [44]:
#1 Question
#For our first foray into aggregates, we're going to stick to something simple. We want to know how many facilities exist - simply produce a total count.
runSelect("select count(*) from cd.facilities")

count
13


In [45]:
#2 Produce a count of the number of facilities that have a cost to guests of 10 or more.
runSelect("select count(*) from cd.facilities \
where guestcost >= 30")

count
5


In [46]:
#3 Produce a count of the number of recommendations each member has made. Order by member ID.
runSelect("select recommendedby, count(*)  from cd.members where recommendedby is not null \
group by recommendedby \
order by recommendedby")

recommendedby,count
1,5
2,3
3,1
4,2
5,1
6,1
9,2
11,1
13,2
15,1


In [47]:
#4 Produce a list of the total number of slots booked per facility. For now, just produce an output table consisting of facility id and slots, sorted by facility id.
runSelect("select facid, sum(slots) as Total_Slots from cd.bookings \
group by facid \
order by facid")

facid,total_slots
0,1320
1,1278
2,1209
3,830
4,1404
5,228
6,1104
7,908
8,911


In [48]:
#5 Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.
runSelect("select facid, sum(slots) as Total_slots from cd.bookings \
where starttime >= '2012-09-01' \
		and starttime < '2012-10-01' \
    	group by facid \
order by sum(slots)")

facid,total_slots
5,122
3,422
7,426
8,471
6,540
2,570
1,588
0,591
4,648


In [67]:
#6 Produce a list of the total number of slots booked per facility per month in the year of 2012. Produce an output table consisting of facility id and slots, sorted by the id and month.
runSelect("select facid, extract(month from starttime) as month, sum(slots) as total from cd.bookings \
where extract(year from starttime) = 2012 \
group by facid, month \
order by facid, month")

facid,month,total
0,7,270
0,8,459
0,9,591
1,7,207
1,8,483
1,9,588
2,7,180
2,8,459
2,9,570
3,7,104


In [76]:
#7Find the total number of members (including guests) who have made at least one booking.
runSelect("select count(distinct memid) from cd.bookings  ")

count
30


In [77]:
#8 Produce a list of facilities with more than 1000 slots booked. Produce an output table consisting of facility id and slots, sorted by facility id.
runSelect("select facid, sum(slots) as Total_slots \
from cd.bookings \
group by facid having sum(slots) > 1000 \
order by facid ")

facid,total_slots
0,1320
1,1278
2,1209
4,1404
6,1104


In [78]:
runSelect("select * from cd.facilities limit 10")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
2,Badminton Court,0,15.5,4000,50
3,Table Tennis,0,5,320,10
4,Massage Room 1,35,80,4000,3000
5,Massage Room 2,35,80,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0,5,450,15
8,Pool Table,0,5,400,15
9,Spa,20,30,100000,800
10,Spa,20,30,100000,800


In [82]:
#9 Produce a list of facilities along with their total revenue. The output table should consist of facility name and revenue, sorted by revenue. Remember that there's a different cost for guests and members!
runSelect("select facs.name, sum(slots * case when memid = 0 then facs.guestcost else facs.membercost end) as revenue \
from cd.bookings bks \
inner join cd.facilities facs \
on bks.facid = facs.facid \
	group by facs.name \
order by revenue ")

name,revenue
Table Tennis,180
Snooker Table,240
Pool Table,270
Badminton Court,1906.5
Squash Court,13468.0
Tennis Court 1,13860
Tennis Court 2,14310
Massage Room 2,15810
Massage Room 1,72540


In [84]:
#10 Produce a list of facilities with a total revenue less than 1000. Produce an output table consisting of facility name and revenue, sorted by revenue. Remember that there's a different cost for guests and members!
runSelect("select name, revenue from (select facs.name, sum(slots * case when memid = 0 then facs.guestcost else facs.membercost end) as revenue \
from cd.bookings bks \
inner join cd.facilities facs on bks.facid = facs.facid \
	group by facs.name \
	) as agg where revenue < 1000 \
  order by revenue")

name,revenue
Table Tennis,180
Snooker Table,240
Pool Table,270


In [88]:
#11 Output the facility id that has the highest number of slots booked. For bonus points, try a version without a LIMIT clause. This version will probably look messy!
runSelect("select facid, sum(slots) as slots from cd.bookings \
group by facid \
order by sum(slots) desc limit 1")



facid,slots
4,1404


In [34]:
#12 Produce a list of the total number of slots booked per facility per month in the year of 2012. In this version, include output rows containing totals for all months per facility, and a total for all months for all facilities.
#The output table should consist of facility id, month and slots, sorted by the id and month. When calculating the aggregated values for all months and all facids, return null values in the month and facid columns.
runSelect("select facid, extract(month from starttime) as month, sum(slots) as slots \
	from cd.bookings \
	where \
		starttime >= '2012-01-01' \
		and starttime < '2013-01-01' \
	group by rollup(facid, month) \
order by facid, month; ")

facid,month,slots
0.000000,7,270
0.000000,8,459
0.000000,9,591
0.000000,None,1320
1.000000,7,207
1.000000,8,483
1.000000,9,588
1.000000,None,1278
2.000000,7,180
2.000000,8,459


In [35]:
#13 Produce a list of the total number of hours booked per facility, remembering that a slot lasts half an hour. The output table should consist of the facility id, name, and hours booked, sorted by facility id. Try formatting the hours to two decimal places.
runSelect("select facs.facid, facs.name, \
	trim(to_char(sum(bks.slots)/2.0, '9999999999999999D99')) as Total_Hours \
	from cd.bookings bks \
	inner join cd.facilities facs \
		on facs.facid = bks.facid \
	group by facs.facid, facs.name \
order by facs.facid")

facid,name,total_hours
0,Tennis Court 1,660.00
1,Tennis Court 2,639.00
2,Badminton Court,604.50
3,Table Tennis,415.00
4,Massage Room 1,702.00
5,Massage Room 2,114.00
6,Squash Court,552.00
7,Snooker Table,454.00
8,Pool Table,455.50


In [36]:
#14 Produce a list of each member name, id, and their first booking after September 1st 2012. Order by member ID.
runSelect("select mems.surname, mems.firstname, mems.memid, min(bks.starttime) as starttime \
	from cd.bookings bks \
	inner join cd.members mems on \
		mems.memid = bks.memid \
	where starttime >= '2012-09-01' \
	group by mems.surname, mems.firstname, mems.memid \
order by mems.memid ")

surname,firstname,memid,starttime
GUEST,GUEST,0,2012-09-01 08:00:00
Smith,Darren,1,2012-09-01 09:00:00
Smith,Tracy,2,2012-09-01 11:30:00
Rownam,Tim,3,2012-09-01 16:00:00
Joplette,Janice,4,2012-09-01 15:00:00
Butters,Gerald,5,2012-09-02 12:30:00
Tracy,Burton,6,2012-09-01 15:00:00
Dare,Nancy,7,2012-09-01 12:30:00
Boothe,Tim,8,2012-09-01 08:30:00
Stibbons,Ponder,9,2012-09-01 11:00:00


In [37]:
#15 Produce a list of member names, with each row containing the total member count. Order by join date, and include guest members.
runSelect("select (select count(*) from cd.members) as count, firstname, surname \
	from cd.members \
order by joindate")

count,firstname,surname
31,GUEST,GUEST
31,Darren,Smith
31,Tracy,Smith
31,Tim,Rownam
31,Janice,Joplette
31,Gerald,Butters
31,Burton,Tracy
31,Nancy,Dare
31,Tim,Boothe
31,Ponder,Stibbons


In [38]:
#16 Produce a monotonically increasing numbered list of members (including guests), ordered by their date of joining. Remember that member IDs are not guaranteed to be sequential.
runSelect("select row_number() over(order by joindate), firstname, surname \
	from cd.members \
order by joindate ")

row_number,firstname,surname
1,GUEST,GUEST
2,Darren,Smith
3,Tracy,Smith
4,Tim,Rownam
5,Janice,Joplette
6,Gerald,Butters
7,Burton,Tracy
8,Nancy,Dare
9,Tim,Boothe
10,Ponder,Stibbons


In [42]:
#17 Output the facility id that has the highest number of slots booked. Ensure that in the event of a tie, all tieing results get output.
runSelect("select facid, sum(slots) as totalslots from cd.bookings \
group by facid \
having sum(slots) = (select max(sum2.totalslots) from \
		(select sum(slots) as totalslots \
		from cd.bookings \
		group by facid \
		) as sum2)")

facid,totalslots
4,1404


facid,totalslots
4,1404


In [43]:
#18 Produce a list of members (including guests), along with the number of hours they've booked in facilities, rounded to the nearest ten hours. Rank them by this rounded figure, producing output of first name, surname, rounded hours, rank. Sort by rank, surname, and first name.
runSelect("select firstname, surname, \
	((sum(bks.slots)+10)/20)*10 as hours, \
	rank() over (order by ((sum(bks.slots)+10)/20)*10 desc) as rank \
	from cd.bookings bks \
	inner join cd.members mems \
		on bks.memid = mems.memid \
	group by mems.memid \
order by rank, surname, firstname ")

firstname,surname,hours,rank
GUEST,GUEST,1200,1
Darren,Smith,340,2
Tim,Rownam,330,3
Tim,Boothe,220,4
Tracy,Smith,220,4
Gerald,Butters,210,6
Burton,Tracy,180,7
Charles,Owen,170,8
Janice,Joplette,160,9
Anne,Baker,150,10


In [44]:
#19 Produce a list of the top three revenue generating facilities (including ties). Output facility name and rank, sorted by rank and facility name.
runSelect("select name, rank from ( \
	select facs.name as name, rank() over (order by sum(case \
				when memid = 0 then slots * facs.guestcost \
				else slots * membercost \
			end) desc) as rank \
		from cd.bookings bks \
		inner join cd.facilities facs \
			on bks.facid = facs.facid \
		group by facs.name \
	) as subq \
	where rank <= 3 \
order by rank")

name,rank
Massage Room 1,1
Massage Room 2,2
Tennis Court 2,3


In [45]:
#20 Classify facilities into equally sized groups of high, average, and low based on their revenue. Order by classification and facility name.
runSelect("select name, case when class=1 then 'high' \
		when class=2 then 'average' \
		else 'low' \
		end revenue \
	from ( \
		select facs.name as name, ntile(3) over (order by sum(case \
				when memid = 0 then slots * facs.guestcost \
				else slots * membercost \
			end) desc) as class \
		from cd.bookings bks \
		inner join cd.facilities facs \
			on bks.facid = facs.facid \
		group by facs.name \
	) as subq \
order by class, name ")

name,revenue
Massage Room 1,high
Massage Room 2,high
Tennis Court 2,high
Badminton Court,average
Squash Court,average
Tennis Court 1,average
Pool Table,low
Snooker Table,low
Table Tennis,low


In [46]:
#21 Based on the 3 complete months of data so far, calculate the amount of time each facility will take to repay its cost of ownership. Remember to take into account ongoing monthly maintenance. Output facility name and payback time in months, order by facility name. Don't worry about differences in month lengths, we're only looking for a rough value here!
runSelect("select 	facs.name as name, \
	facs.initialoutlay/((sum(case \
			when memid = 0 then slots * facs.guestcost \
			else slots * membercost \
		end)/3) - facs.monthlymaintenance) as months \
	from cd.bookings bks \
	inner join cd.facilities facs \
		on bks.facid = facs.facid \
	group by facs.facid \
order by name")

name,months
Badminton Court,6.8317677198975235
Massage Room 1,0.18885741265344664778
Massage Room 2,1.7621145374449339
Pool Table,5.3333333333333333
Snooker Table,6.9230769230769231
Squash Court,1.1339582703356516
Table Tennis,6.4000000000000000
Tennis Court 1,2.2624434389140271
Tennis Court 2,1.7505470459518600


In [68]:
#22 For each day in August 2012, calculate a rolling average of total revenue over the previous 15 days. Output should contain date and revenue columns, sorted by the date. Remember to account for the possibility of a day having zero revenue. This one's a bit tough, so don't be afraid to check out the hint!
runSelect("select 	dategen.date, \
	        (select sum(case \
        			when memid = 0 then slots * facs.guestcost \
        			else slots * membercost \
      		end) as rev from cd.bookings bks \
          		inner join cd.facilities facs on bks.facid = facs.facid \
              where bks.starttime > dategen.date - interval '14 days' \
        			and bks.starttime < dategen.date + interval '1 day')/15 as revenue \
        	from (select 	cast(generate_series(timestamp '2012-08-01', '2012-08-31','1 day') as date) as date)  as dategen \
          order by dategen.date")

date,revenue
2012-08-01,1126.8333333333333333
2012-08-02,1153.0000000000000000
2012-08-03,1162.9000000000000000
2012-08-04,1177.3666666666666667
2012-08-05,1160.9333333333333333
2012-08-06,1185.4000000000000000
2012-08-07,1182.8666666666666667
2012-08-08,1172.6000000000000000
2012-08-09,1152.4666666666666667
2012-08-10,1175.0333333333333333


#Date

In [50]:
#1 Produce a timestamp for 1 a.m. on the 31st of August 2012
runSelect("select timestamp '2012-08-31 01:00:00'")

timestamp
2012-08-31 01:00:00


In [59]:
#2 Find the result of subtracting the timestamp '2012-07-30 01:00:00' from the timestamp '2012-08-31 01:00:00'
runSelect("select timestamp '2012-08-31 01:00:00' - timestamp '2012-07-30 01:00:00' as Interval")

interval
32 days 00:00:00


In [53]:
#3 Produce a list of all the dates in October 2012. They can be output as a timestamp (with time set to midnight) or a date.
runSelect("select generate_series(timestamp '2012-10-01', timestamp '2012-10-31', interval '1 day') ts")

ts
2012-10-01 00:00:00
2012-10-02 00:00:00
2012-10-03 00:00:00
2012-10-04 00:00:00
2012-10-05 00:00:00
2012-10-06 00:00:00
2012-10-07 00:00:00
2012-10-08 00:00:00
2012-10-09 00:00:00
2012-10-10 00:00:00


In [55]:
#4 Get the day of the month from the timestamp '2012-08-31' as an integer.
runSelect("select extract(day from timestamp '2012-08-31')")



extract
31


In [56]:
#5 Work out the number of seconds between the timestamps '2012-08-31 01:00:00' and '2012-09-02 00:00:00'
runSelect("select extract(epoch from (timestamp '2012-09-02 00:00:00' - '2012-08-31 01:00:00'))")

extract
169200.000000


In [62]:
#6 For each month of the year in 2012, output the number of days in that month. Format the output as an integer column containing the month of the year, and a second column containing an interval data type.
runSelect("select extract(month from cal.month) as month, \
(cal.month + interval '1 month') - cal.month as length \
from (select generate_series(timestamp '2012-01-01', timestamp '2012-12-01', interval '1 month') as month) cal \
order by month")


month,length
1,31 days 00:00:00
2,29 days 00:00:00
3,31 days 00:00:00
4,30 days 00:00:00
5,31 days 00:00:00
6,30 days 00:00:00
7,31 days 00:00:00
8,31 days 00:00:00
9,30 days 00:00:00
10,31 days 00:00:00


In [59]:
#7 For any given timestamp, work out the number of days remaining in the month. The current day should count as a whole day, regardless of the time. Use '2012-02-11 01:00:00' as an example timestamp for the purposes of making the answer. Format the output as a single interval value.
runSelect("select (date_trunc('month',ts.testts) + interval '1 month') \
		- date_trunc('day', ts.testts)  remaining \
	from (select timestamp '2012-02-11 01:00:00' as testts) ts ")

remaining
19 days 00:00:00


In [60]:
#8 Return a list of the start and end time of the last 10 bookings (ordered by the time at which they end, followed by the time at which they start) in the system
runSelect("select starttime, starttime + slots*(interval '30 minutes') endtime \
	from cd.bookings \
	order by endtime desc, starttime desc \
	limit 10   ")

starttime,endtime
2013-01-01 15:30:00,2013-01-01 16:00:00
2012-09-30 19:30:00,2012-09-30 20:30:00
2012-09-30 19:00:00,2012-09-30 20:30:00
2012-09-30 19:30:00,2012-09-30 20:00:00
2012-09-30 19:00:00,2012-09-30 20:00:00
2012-09-30 19:00:00,2012-09-30 20:00:00
2012-09-30 18:30:00,2012-09-30 20:00:00
2012-09-30 18:30:00,2012-09-30 20:00:00
2012-09-30 19:00:00,2012-09-30 19:30:00
2012-09-30 18:30:00,2012-09-30 19:30:00


In [61]:
#9 Return a count of bookings for each month, sorted by month
runSelect("select date_trunc('month', starttime) as month, count(*) \
	from cd.bookings \
	group by month \
	order by month ")

month,count
2012-07-01 00:00:00,658
2012-08-01 00:00:00,1472
2012-09-01 00:00:00,1913
2013-01-01 00:00:00,1


In [62]:
#10 Work out the utilisation percentage for each facility by month, sorted by name and month, rounded to 1 decimal place. Opening time is 8am, closing time is 8.30pm. You can treat every month as a full month, regardless of if there were some dates the club was not open.
runSelect("select name, month, \
	round((100*slots)/ \
		cast( 25*(cast((month + interval '1 month') as date) - cast (month as date)) as numeric),1) utilisation \
	from  (select facs.name as name, date_trunc('month', starttime) as month, sum(slots) slots \
			from cd.bookings bks \
			inner join cd.facilities facs \
				on bks.facid = facs.facid \
			group by facs.facid, month) inn \
order by name, month ")

name,month,utilisation
Badminton Court,2012-07-01 00:00:00,23.2
Badminton Court,2012-08-01 00:00:00,59.2
Badminton Court,2012-09-01 00:00:00,76.0
Massage Room 1,2012-07-01 00:00:00,34.1
Massage Room 1,2012-08-01 00:00:00,63.5
Massage Room 1,2012-09-01 00:00:00,86.4
Massage Room 2,2012-07-01 00:00:00,3.1
Massage Room 2,2012-08-01 00:00:00,10.6
Massage Room 2,2012-09-01 00:00:00,16.3
Pool Table,2012-07-01 00:00:00,15.1


# String Operations

In [63]:
#1 Output the names of all members, formatted as 'Surname, Firstname'
runSelect("select surname || ', ' || firstname as name from cd.members ")

name
"GUEST, GUEST"
"Smith, Darren"
"Smith, Tracy"
"Rownam, Tim"
"Joplette, Janice"
"Butters, Gerald"
"Tracy, Burton"
"Dare, Nancy"
"Boothe, Tim"
"Stibbons, Ponder"


In [67]:
#2 Find all facilities whose name begins with 'Tennis'. Retrieve all columns.
runSelect("select * from cd.facilities where name like 'Tennis%'")


facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,10000,200


In [68]:
#3 Perform a case-insensitive search to find all facilities whose name begins with 'tennis'. Retrieve all columns.
runSelect("select * from cd.facilities where upper(name) like 'TENNIS%';  ")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,10000,200


In [69]:
#4 You've noticed that the club's member table has telephone numbers with very inconsistent formatting. You'd like to find all the telephone numbers that contain parentheses, returning the member ID and telephone number sorted by member ID.
runSelect("select memid, telephone from cd.members where telephone ~ '[()]' ")

memid,telephone
0,(000) 000-0000
3,(844) 693-0723
4,(833) 942-4710
5,(844) 078-4130
6,(822) 354-9973
7,(833) 776-4001
8,(811) 433-2547
9,(833) 160-3900
10,(855) 542-5251
11,(844) 536-8036


In [70]:
#5 The zip codes in our example dataset have had leading zeroes removed from them by virtue of being stored as a numeric type. Retrieve all zip codes from the members table, padding any zip codes less than 5 characters long with leading zeroes. Order by the new zip code.
runSelect("select lpad(cast(zipcode as char(5)),5,'0') zip from cd.members order by zip")

zip
00000
00234
00234
04321
04321
10383
11986
23423
28563
33862


In [71]:
#6 You'd like to produce a count of how many members you have whose surname starts with each letter of the alphabet. Sort by the letter, and don't worry about printing out a letter if the count is 0.
runSelect("select substr (mems.surname,1,1) as letter, count(*) as count \
    from cd.members mems \
    group by letter \
    order by letter ")

letter,count
B,5
C,2
D,1
F,2
G,2
H,1
J,3
M,1
O,1
P,2


In [72]:
#7 The telephone numbers in the database are very inconsistently formatted. You'd like to print a list of member ids and numbers that have had '-','(',')', and ' ' characters removed. Order by member id.
runSelect("select memid, translate(telephone, '-() ', '') as telephone from cd.members order by memid")

memid,telephone
0,0000000000
1,5555555555
2,5555555555
3,8446930723
4,8339424710
5,8440784130
6,8223549973
7,8337764001
8,8114332547
9,8331603900


# Recursive Queries

In [56]:
#1
runSelect("with recursive recommenders(recommender) as (select recommendedby from cd.members where memid = 27 \
	union all \
	select mems.recommendedby \
		from recommenders recs \
		inner join cd.members mems \
			on mems.memid = recs.recommender) \
select recs.recommender, mems.firstname, mems.surname \
	from recommenders recs \
	inner join cd.members mems \
		on recs.recommender = mems.memid \
order by memid desc")

recommender,firstname,surname
20,Matthew,Genting
5,Gerald,Butters
1,Darren,Smith


In [57]:
#2 Find the downward recommendation chain for member ID 1: that is, the members they recommended, the members those members recommended, and so on. Return member ID and name, and order by ascending member id.
runSelect("with recursive recommendeds(memid) as (select memid from cd.members where recommendedby = 1 \
	union all \
	select mems.memid \
		from recommendeds recs \
		inner join cd.members mems \
			on mems.recommendedby = recs.memid) \
select recs.memid, mems.firstname, mems.surname \
	from recommendeds recs \
	inner join cd.members mems \
		on recs.memid = mems.memid \
order by memid ")

memid,firstname,surname
4,Janice,Joplette
5,Gerald,Butters
7,Nancy,Dare
10,Charles,Owen
11,David,Jones
14,Jack,Smith
20,Matthew,Genting
21,Anna,Mackenzie
26,Douglas,Jones
27,Henrietta,Rumney


In [58]:
#3 Produce a CTE that can return the upward recommendation chain for any member. You should be able to select recommender from recommenders where member=x. Demonstrate it by getting the chains for members 12 and 22. Results table should have member and recommender, ordered by member ascending, recommender descending.
runSelect("with recursive recommenders(recommender, member) as ( \
	select recommendedby, memid \
		from cd.members \
	union all \
	select mems.recommendedby, recs.member \
		from recommenders recs \
		inner join cd.members mems \
			on mems.memid = recs.recommender) \
select recs.member member, recs.recommender, mems.firstname, mems.surname \
	from recommenders recs \
	inner join cd.members mems on recs.recommender = mems.memid \
	where recs.member = 22 or recs.member = 12 \
order by recs.member asc, recs.recommender desc ")

member,recommender,firstname,surname
12,9,Ponder,Stibbons
12,6,Burton,Tracy
22,16,Timothy,Baker
22,13,Jemima,Farrell
